# Introduction

*Note: For the text answers in the notebook, you can either write them in a cell in the notebook, or write them on paper and submit them as a photo/pdf along with the notebook.*

## Solving a small linear problem

Before moving on to the main topic of this lab, which will be the transportation problem, to get you started, we will go through coding a small linear program using OR-Tools. There is also an accompanying pdf document that goes over the basics of OR-Tools in a bit more detail, using max-flow as an example. The linear program the code bellow implements is the following:

<center>
$
\begin{align}
\max&\ 2x_1 + 2x_2 + 3x_3\\
\text{s.t.}&\ 2x_1 + 7x_2 + 3x_3 \le 50\\
& 3x_1 - 5x_2 + 7x_3 \le 45\\
& 5x_1 + 2x_2 - 6x_3 \le 37\\
& x_1,x_2,x_3\ge 0
\end{align}
$
    
    
In the comments before each line of code, we explain which part of the program is being implemented. 

In [1]:
### Coding a linear program
# Importing the solver that we will use for modeling/solving from OR-Tools
from ortools.linear_solver import pywraplp

# We initliaze a variable to store the solver that we will use
# The name of the solver is GLOP
solver = pywraplp.Solver.CreateSolver('GLOP')

# Creating the three variables x1,x2,x3
# initialize them as real variables that are greater or equal to 0(the NumVar method does that)
# we also create a dictionary, and in each entry we store on of the variables, using the 
# indices as keys
x = {}
for i in range(1,4):
    x[i] = solver.NumVar(0, solver.infinity(), 'x'+str(i))

print('Number of variables =', solver.NumVariables())

# Constraint: 2x_1 + 7x_2 + 3x_3 <= 50
solver.Add(2*x[1] + 7*x[2] + 3*x[3] <= 50)

# Constraint: 3x_1 - 5x_2 + 7x_3 <= 45
solver.Add(3*x[1] - 5*x[2] + 7*x[3] <= 45)

# Constraint: 5x_1 + 2x_2 - 6x_3 <= 37
solver.Add(5*x[1] + 2*x[2] - 6*x[3] <= 37)

print('Number of constraints =', solver.NumConstraints())

# Objective function: maximize 2x_1 + 2x_2 + 3x_3
solver.Maximize(2*x[1] + 2*x[2] + 3*x[3])

# Solve the model created 
status = solver.Solve()

# Print some information about the solution 
print('Solution:')
print('Objective value =', solver.Objective().Value())
for i in range(1,4):
    print('x'+str([i])+ '= ', x[i].solution_value())


Number of variables = 3
Number of constraints = 3
Solution:
Objective value = 37.39919354838709
x[1]=  10.741935483870966
x[2]=  2.5201612903225805
x[3]=  3.6249999999999996


## The Transportation Problem
<br>
In this lab, we will be introducing Google OR-Tools for python, which is an open source software package that we will be using throughout the semester to solve various optimization problems. 
<br> <br>
For today, our focus will be on the transportation problem. In this problem, we are given a set of facilities, and our goal is to move a commodity between them, while minimizing the cost of moving those commodities. We will distinguish two types of facilities, ones that have commodity available to move to other facilities, which will be called sources, and the ones that require commodity shipped to them, which will be called destinations. In more detail, we are given:

- A set $I$ of $i = 1,\dots,m$ sources, along with supply of commodities available at source $i$, which will be denoted by $s_i$.
- A set $J$ of $j = 1,\dots,n$ destinations, along with the demand of commodities required at destination $j$, which will be denoted by $d_j$.
- The cost $c_{ij}$ of moving one unit of commodity from source $i$ to destination $j$.

Our goal is to move goods, so that all demands on the destinations are met, using the ammount of commodity available at sources, while minimizing the total transportation cost. One assumption that we will make, is that the total quantity of commodity available across all sources is equal to the total demand across all destinations.

In [2]:
# imports
import pandas as pd

Run the cell bellow to read and print an example set of data that we will use to create and solve a transportation problem model. The *Demand* row gives the amount of commodity requested at each warehouse. And the *Supply* column gives the amount of commodity available at each factory. The rest of the cells give the cost of shipping one unit of commodity from each factory to each warehouse.

In [3]:
example_data = pd.read_csv('instance1.csv', index_col = 0)
display(example_data)

,Warehouse1,Warehouse2,Warehouse3,Warehouse4,Supply
Sources,,,,,
Factory1,12,12,15,7,50
Factory2,13,11,10,13,70
Factory3,9,6,4,12,30
Factory4,8,13,8,9,50
Demand,25,35,105,35,200


Write down the linear program that corresponds to the transportation problem instance for the given data.

#### Answer:

\begin{equation}
\begin{aligned}
& \text{Minimize} & & \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij} \cdot x_{ij} \\
& \text{Subject to} & & \\
& & & \sum_{j=1}^{n} x_{ij} \leq s_i & \text{for } i = 1, \ldots, m \\
& & & \sum_{i=1}^{m} x_{ij} \geq d_j & \text{for } j = 1, \ldots, n \\
& & & x_{ij} \geq 0 & \text{for all } i, j
\end{aligned}
\end{equation}


Let us now move on to creating and solving a model that uses this set of data. To do so, we will create a linear program that describes this transportation problem. We will go through the steps required to formulate and solve a linear program using Google OR-Tools, while also  giving a mathematical description of the step that we are implementing. <br> <br> 

In total, we will go through the following steps:
 - Create the variables.
 - Define the constraints.
 - Define the objective function.
 - Calling the solver and printing the results.
<br>

 
Before that, run the following cell to import OR-Tools and declare the solver that we will be using to solve the linear problem. 'GLOP' is the name of the solver we will be using, and while there are other solvers available, we will not be going through them now.

In [11]:
from ortools.linear_solver import pywraplp as ORTools
lp = ORTools.Solver.CreateSolver('GLOP')

##### Creating the Variables

Given the set $I$ of sources and set $J$ of destinations, we will define integer variables $x_{ij}, \forall i \in I, j \in J$, to denote the amount of commodity shipped from source $i$ to destination $j$. Since these variables describe the amount of commodity, we will have $x_{ij} \ge 0, \forall i \in I, j \in J$. Run the cell bellow to create the sets $I$ and $J$ from the given input data.

In [6]:
# notice we exclude the last row/column in each case, since it is the one name Demand/Supply
I = list(example_data.index)[:-1]
print(I)
J = list(example_data.columns)[:-1]
print(J)

['Factory1', 'Factory2', 'Factory3', 'Factory4']
['Warehouse1', 'Warehouse2', 'Warehouse3', 'Warehouse4']


Now we can create the integer variables $x_{ij}$. The function [`IntVar(lb,ub,name)`](https://developers.google.com/optimization/reference/python/linear_solver/pywraplp?hl=en#intvar) can be used to add variables to the model, where `lb, ub` are the lower and upper bounds of the variable, and `name` the name of the variable in the OR-Tools model. 
In the cell bellow, loop over the sets $I$ and $J$ and add the variables $x_{ij}$ to the model, storing them in dictionary `x`. If a variable does not have an upper bound, you should use `lp.infinity()`. 

In [12]:
# Import OR-Tools and declare the solver
from ortools.linear_solver import pywraplp as ORTools
lp = ORTools.Solver.CreateSolver('GLOP')

# Create sets I and J
I = list(example_data.index)[:-1]
J = list(example_data.columns)[:-1]

# Create the integer variables x_ij
x = {}  # dictionary to store the variables x_ij

for i in I:
    for j in J:
        x[i, j] = lp.IntVar(0, lp.infinity(), f'x_{i}_{j}')

# Display the variables
display(x)


{('Factory1', 'Warehouse1'): x_Factory1_Warehouse1,
 ('Factory1', 'Warehouse2'): x_Factory1_Warehouse2,
 ('Factory1', 'Warehouse3'): x_Factory1_Warehouse3,
 ('Factory1', 'Warehouse4'): x_Factory1_Warehouse4,
 ('Factory2', 'Warehouse1'): x_Factory2_Warehouse1,
 ('Factory2', 'Warehouse2'): x_Factory2_Warehouse2,
 ('Factory2', 'Warehouse3'): x_Factory2_Warehouse3,
 ('Factory2', 'Warehouse4'): x_Factory2_Warehouse4,
 ('Factory3', 'Warehouse1'): x_Factory3_Warehouse1,
 ('Factory3', 'Warehouse2'): x_Factory3_Warehouse2,
 ('Factory3', 'Warehouse3'): x_Factory3_Warehouse3,
 ('Factory3', 'Warehouse4'): x_Factory3_Warehouse4,
 ('Factory4', 'Warehouse1'): x_Factory4_Warehouse1,
 ('Factory4', 'Warehouse2'): x_Factory4_Warehouse2,
 ('Factory4', 'Warehouse3'): x_Factory4_Warehouse3,
 ('Factory4', 'Warehouse4'): x_Factory4_Warehouse4}

##### Defining the constraints

Recall that in the transportation problem, we want to meet the demand $d_j$ on each destinations $j$, while respecting the supply $s_i$ available at each source $i$. Run the cell bellow to create the dictionaries that will store the supply and demand of every source and destination.

*Note on how the dictionaries in the next cell are created:* The `to_dict` method will create a dictionary from a dataframe column, that has keys the indices and values the corresponding column value. We again exclude the last row/column that has Demand/Supply. In the second case, we take the transpose, since demand is stored as a row.

In [13]:
supply = example_data['Supply'][:-1].to_dict()
print(supply)
demand = example_data.transpose()['Demand'][:-1].to_dict()
print(demand)

{'Factory1': 50, 'Factory2': 70, 'Factory3': 30, 'Factory4': 50}
{'Warehouse1': 25, 'Warehouse2': 35, 'Warehouse3': 105, 'Warehouse4': 35}


Now the constraint that we want to add can be described as:

$$ 
\sum_{j\in J} x_{ij} = s_i, \forall i \in I
$$
and
$$ 
\sum_{i\in I} x_{ij} = d_j, \forall j \in J
$$
where the first one describes the fact that we should use the supply at every source node, and the second one that we should meet the demand in every destination node.
<br><br>
In the cell bellow you can find the code that adds the first constraint, using the [`Add(constraint)`](https://developers.google.com/optimization/reference/python/linear_solver/pywraplp?hl=en#add) method. Use similar code to add the second constraint.

In [15]:
# adds the supply met at sources constraint
for i in I:
    lp.Add(sum(x[i, j] for j in J) == supply[i])

# adds the demand met at destinations constraint
for j in J:
    lp.Add(sum(x[i, j] for i in I) == demand[j])


#### Define the objective function

The final part before solving the model is adding the objective function. Remember that there is cost $c_{ij}$ for moving commodity from source $i$ to destination $j$, and our goal is to minimize the total cost of moving all the goods. Run the cell bellow to create the dictionary `cost` that stores the corresponding costs $c_{ij}$.

In [20]:
# Create the dictionary for costs
cost = example_data.iloc[:-1, :-1].transpose().to_dict()
cost = {(i, j): cost[i][j] for i in cost for j in cost[i]}

# Display the cost dictionary
display(cost)


{('Factory1', 'Warehouse1'): 12,
 ('Factory1', 'Warehouse2'): 12,
 ('Factory1', 'Warehouse3'): 15,
 ('Factory1', 'Warehouse4'): 7,
 ('Factory2', 'Warehouse1'): 13,
 ('Factory2', 'Warehouse2'): 11,
 ('Factory2', 'Warehouse3'): 10,
 ('Factory2', 'Warehouse4'): 13,
 ('Factory3', 'Warehouse1'): 9,
 ('Factory3', 'Warehouse2'): 6,
 ('Factory3', 'Warehouse3'): 4,
 ('Factory3', 'Warehouse4'): 12,
 ('Factory4', 'Warehouse1'): 8,
 ('Factory4', 'Warehouse2'): 13,
 ('Factory4', 'Warehouse3'): 8,
 ('Factory4', 'Warehouse4'): 9}

Since we want to minimize the total cost, the objective function will be:

$$
\min \sum_{i \in I, j \in J} c_{ij}\cdot x_{ij}
$$
use the [`Minimize`](https://developers.google.com/optimization/reference/python/linear_solver/pywraplp?hl=en#minimize) method to add the objective function to the model.

In [22]:
# Minimize the total cost objective function
lp.Minimize(lp.Sum(cost[i, j] * x[i, j] for i in I for j in J))


#### Calling the solver and printing the results

We are now ready to solve the model for the input data given. Running the cell bellow will call the [`Solve()`](https://developers.google.com/optimization/reference/python/linear_solver/pywraplp?hl=en#solve) method to solve the program. The few lines of code after that will print the objective value, as will as the variables that have value greater than 0.

In [23]:
lp.Solve()
print('Objective value =',lp.Objective().Value())
for var in lp.variables():
    if var.solution_value() > 0:
        print(var.name(),round(var.solution_value()))

Objective value = 1665.0
x_Factory1_Warehouse2 15
x_Factory1_Warehouse4 35
x_Factory2_Warehouse2 20
x_Factory2_Warehouse3 50
x_Factory3_Warehouse3 30
x_Factory4_Warehouse1 25
x_Factory4_Warehouse3 25


We are done! By looking at the indices of the variables that are greater than zero, we can see which factory will supply which warehouse, and the amount of goods shipped. In the rest of the labs we will be coding larger models, so understanding the steps that we went throught to implement this integer program is important, as these are the basic steps one follows to code any model. <br> <br>

Putting together all the constraints and objective function, the program we coded is the following:
<center>
$
\begin{align}
\min& \sum_{i \in I, j \in J} c_{ij}\cdot x_{ij}\\
\text{s.t.}& \sum_{j\in J} x_{ij} = s_i, \forall i \in I\\
& \sum_{i\in I} x_{ij} = d_j, \forall j \in J\\
& x_{ij}\ge 0, \text{integer},\ \forall i \in I, j \in J
\end{align}
$
    
Something that will be usefull throughout the course is defining functions that solve programs like this one. In the cell bellow you can find a function that solves the same model you just coded. Try calling the function on the input data to confirm that you get the same solution.     

In [27]:
def transportation_p(input_data):

    lp = ORTools.Solver.CreateSolver('GLOP')
    
    I = list(input_data.index)[:-1]
    J = list(input_data.columns)[:-1]
    x = {} # dictionary to store the variables x_ij 

    for i in I:
        for j in J:
            x[i,j] = lp.IntVar(0,lp.infinity(),'x'+str([i,j]))
            
    for i in I:
        lp.Add(sum(x[i,j] for j in J) == supply[i])
    for j in J:
        lp.Add(sum(x[i,j] for i in I) == demand[j])
        
    cost = example_data.iloc[:-1,:-1].transpose().to_dict()
    cost = {(i,j) : cost[i][j] for i in cost for j in cost[i]} 
    
    lp.Minimize(sum(cost[i,j]*x[i,j] for i in I for j in J))

    lp.Solve()
    print('Objective value =',lp.Objective().Value())
    for var in lp.variables():
        if var.solution_value() > 0:
            print(var.name(),round(var.solution_value()))

In [28]:
# Call the function on the same instance you solved before
transportation_p(example_data)

Objective value = 1665.0
x['Factory1', 'Warehouse2'] 15
x['Factory1', 'Warehouse4'] 35
x['Factory2', 'Warehouse2'] 20
x['Factory2', 'Warehouse3'] 50
x['Factory3', 'Warehouse3'] 30
x['Factory4', 'Warehouse1'] 25
x['Factory4', 'Warehouse3'] 25


### Question 
The transportation problem we just coded is called a *balanced* transportation problem, which means that the total demand available is equal to the total supply available. In other versions of the problem, we can have larger demand than supply, or supply larger than the demand. Suggest some ideas to solve these other versions of the problem.
<br><br>

*Hint:* You can either suggest some change in the input data or edits to the model in order to solve it as an integer program.

#### Answer:
To handle cases where the transportation problem is unbalanced, meaning that the total supply is not equal to the total demand, you can make some adjustments to the input data or the model. Here are some suggestions:

Case: Larger Demand than Supply:

If the demand is greater than the supply, you can introduce a dummy source with a supply equal to the difference between demand and supply. This dummy source will have zero transportation costs to all destinations.
You can add this dummy source to the set of sources (

I) with an appropriate supply value and modify the supply constraints accordingly.
Case: Larger Supply than Demand:

If the supply is greater than the demand, you can introduce a dummy destination with a demand equal to the difference between supply and demand. This dummy destination will have zero transportation costs from all sources.
You can add this dummy destination to the set of destinations (

J) with an appropriate demand value and modify the demand constraints accordingly.
Here's a brief modification to the code to handle the case of larger demand than supply (Case 1):